In [1]:
from docplex.cp.model import *
from docplex.cp.config import get_default

In [2]:
from config import setup
setup()

In [3]:
n = 6

F = ['f1', 'f2', 'f3', 'f4', 'f5', 'f6']
H = ['h1', 'h2', 'h3', 'h4', 'h5', 'h6']
L= {}
pref_h = [
    [1,3,6,2,4,5],
    [4,6,1,2,5,3],
    [1,4,5,3,6,2],
    [6,5,3,4,2,1],
    [2,3,1,4,5,6],
    [3,1,2,6,5,4],
]
pref_f = [
    [1,5,6,3,2,4],
    [2,4,6,1,3,5],
    [4,3,6,2,5,1],
    [1,3,5,4,2,6],
    [3,2,6,1,4,5],
    [5,1,3,6,4,2],
]
for i, hlist in enumerate(pref_h):
    L['h'+str(i+1)]=['f'+str(j) for j in hlist]
for i, flist in enumerate(pref_f):
    L['f'+str(i+1)]=['h'+str(j) for j in flist]


mdl = CpoModel(name="Mariages")
L

{'h1': ['f1', 'f3', 'f6', 'f2', 'f4', 'f5'],
 'h2': ['f4', 'f6', 'f1', 'f2', 'f5', 'f3'],
 'h3': ['f1', 'f4', 'f5', 'f3', 'f6', 'f2'],
 'h4': ['f6', 'f5', 'f3', 'f4', 'f2', 'f1'],
 'h5': ['f2', 'f3', 'f1', 'f4', 'f5', 'f6'],
 'h6': ['f3', 'f1', 'f2', 'f6', 'f5', 'f4'],
 'f1': ['h1', 'h5', 'h6', 'h3', 'h2', 'h4'],
 'f2': ['h2', 'h4', 'h6', 'h1', 'h3', 'h5'],
 'f3': ['h4', 'h3', 'h6', 'h2', 'h5', 'h1'],
 'f4': ['h1', 'h3', 'h5', 'h4', 'h2', 'h6'],
 'f5': ['h3', 'h2', 'h6', 'h1', 'h4', 'h5'],
 'f6': ['h5', 'h1', 'h3', 'h6', 'h4', 'h2']}

In [4]:
#Lpos[h,f] postition de f dans L(h) + position de h dans L(h)
LPos = {}
for h in H:
    for f in F:
        LPos[h,f]=L[h].index(f)
        LPos[f,h]=L[f].index(h)


In [5]:
#x[h,f] mariage entre h et f
x = {}
z = {}
for h in H:
    for f in F:
        x[h,f]=mdl.binary_var(name="x_" + h + "_" + f)
        z[h,f]=mdl.binary_var(name="z_" + h + "_" + f)

#y[i] cout du marriage de i (somme des couts du mariage pour les 2 partis) 
y = {}
for i in H+F:
    y[i]=mdl.integer_var(0,n-1,name="y_"+i)




In [6]:
#Ajout des contraintes

#Contrainte de monogamie
for h in H:
    mdl.add(sum([x[h,f] for f in F])==1)
for f in F:
    mdl.add(sum([x[h,f] for h in H])==1)


#Valeur de y (Position du partenaire dans le classement):
for h in H:
    mdl.add(y[h]==sum([(x[h,f]*LPos[h,f]) for f in F]))
for f in F:
    mdl.add(y[f]==sum([(x[h,f]*LPos[f,h]) for h in H]))

In [16]:
#Couple bloquant =
#si (h,f) est un couple bloquant si 
#   - le partenaire actuel de h et en plus haute postion de preference que f
#   - et le partenaire actuel de f et en plus haute postion de preference que h
for h in H:
    for f in F:
        mdl.add(mdl.if_then((y[h]>LPos[h,f]) & (y[f]>LPos[f,h]), z[h,f] == 1))

mdl.add(sum(z[h,f] for h in H for f in F)==0)

In [8]:
#Couple bloquant =
#si (h,f) est un couple bloquant si 
#   - le partenaire actuel de h et en plus haute postion de preference que f
#   - et le partenaire actuel de f et en plus haute postion de preference que h
for h in H:
    for f in F:
        mdl.add((y[h]>LPos[h,f]) & (y[f]>LPos[f,h]) == false())
#mdl.add(sum(z[h,f] for h in H for f in F)==0)

In [9]:
lsol = mdl.start_search(SearchType="DepthFirst",trace_log=False)


In [10]:
#Affichage des solutions:
for solution in lsol:
    print("\n   NOUVELLE SOLUTION :\n")
    for h in H:
        for f in F:
            if(solution[x[h,f]]==1):
                print("Mariage entre ",h," et ",f," (Couts = ",LPos[h,f],LPos[f,h],")")



   NOUVELLE SOLUTION :

Mariage entre  h1  et  f1  (Couts =  0 0 )
Mariage entre  h2  et  f2  (Couts =  3 0 )
Mariage entre  h3  et  f4  (Couts =  1 1 )
Mariage entre  h4  et  f6  (Couts =  0 4 )
Mariage entre  h5  et  f5  (Couts =  4 5 )
Mariage entre  h6  et  f3  (Couts =  0 2 )

   NOUVELLE SOLUTION :

Mariage entre  h1  et  f1  (Couts =  0 0 )
Mariage entre  h2  et  f2  (Couts =  3 0 )
Mariage entre  h3  et  f4  (Couts =  1 1 )
Mariage entre  h4  et  f3  (Couts =  2 0 )
Mariage entre  h5  et  f6  (Couts =  5 0 )
Mariage entre  h6  et  f5  (Couts =  4 2 )

   NOUVELLE SOLUTION :

Mariage entre  h1  et  f1  (Couts =  0 0 )
Mariage entre  h2  et  f2  (Couts =  3 0 )
Mariage entre  h3  et  f4  (Couts =  1 1 )
Mariage entre  h4  et  f5  (Couts =  1 4 )
Mariage entre  h5  et  f6  (Couts =  5 0 )
Mariage entre  h6  et  f3  (Couts =  0 2 )
